In [1]:
!pip install openai num2words matplotlib plotly scipy scikit-learn pandas tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=3f74faebe2f3e7df3f0e16260d2ebbb950f7a485333e7faf108bd1139efd860b
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
!curl "https://raw.githubusercontent.com/Azure-Samples/Azure-OpenAI-Docs-Samples/main/Samples/Tutorials/Embeddings/data/bill_sum_data.csv" --output bill_sum_data.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  232k  100  232k    0     0   932k      0 --:--:-- --:--:-- --:--:--  934k


In [6]:
import os

os.environ['AZURE_OPENAI_API_KEY'] = 'e32de39525d342acb842db5aeb853711'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://cytotropism.openai.azure.com/'

In [7]:
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
import tiktoken
from openai import AzureOpenAI

In [8]:
df=pd.read_csv(os.path.join(os.getcwd(),'bill_sum_data.csv')) # This assumes that you have placed the bill_sum_data.csv in the same directory you are running Jupyter Notebooks
df

,Unnamed: 0,bill_id,text,summary,title,text_len,sum_len
0,0,110_hr37,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,8494,321
1,1,112_hr2873,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,6522,1424
2,2,109_s2408,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,6154,463
3,3,108_s1899,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,19853,1400
4,4,107_s1531,SECTION 1. SHORT TITLE.\n\n This Act may be...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,6273,278
5,5,107_hr4541,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...,11691,114
6,6,111_s1495,SECTION 1. SHORT TITLE.\n\n This Act may be...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...,5328,379
7,7,111_s3885,SECTION 1. SHORT TITLE.\n\n This Act may be...,Race to the Top Act of 2010 - Directs the Secr...,A bill to provide incentives for States and lo...,16668,1525
8,8,113_hr1796,SECTION 1. SHORT TITLE.\n\n This Act may be...,Troop Talent Act of 2013 - Directs the Secreta...,Troop Talent Act of 2013,15352,2151
9,9,103_hr1987,SECTION 1. SHORT TITLE.\n\n This Act may be...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,5633,894


In [9]:
df_bills = df[['text', 'summary', 'title']]
df_bills

,text,summary,title
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...
3,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...
5,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...
6,SECTION 1. SHORT TITLE.\n\n This Act may be...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...
7,SECTION 1. SHORT TITLE.\n\n This Act may be...,Race to the Top Act of 2010 - Directs the Secr...,A bill to provide incentives for States and lo...
8,SECTION 1. SHORT TITLE.\n\n This Act may be...,Troop Talent Act of 2013 - Directs the Secreta...,Troop Talent Act of 2013
9,SECTION 1. SHORT TITLE.\n\n This Act may be...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993


In [10]:
pd.options.mode.chained_assignment = None #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#evaluation-order-matters

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()

    return s

df_bills['text']= df_bills["text"].apply(lambda x : normalize_text(x))

In [11]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df_bills['n_tokens'] = df_bills["text"].apply(lambda x: len(tokenizer.encode(x)))
df_bills = df_bills[df_bills.n_tokens<8192]
len(df_bills)

20

In [12]:
df_bills

,text,summary,title,n_tokens
0,SECTION 1. SHORT TITLE. This Act may be cited ...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,1466
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,1183
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,937
3,SECTION 1. SHORT TITLE. This Act may be cited ...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,3670
4,SECTION 1. SHORT TITLE. This Act may be cited ...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,1038
5,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...,2026
6,SECTION 1. SHORT TITLE. This Act may be cited ...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...,880
7,SECTION 1. SHORT TITLE. This Act may be cited ...,Race to the Top Act of 2010 - Directs the Secr...,A bill to provide incentives for States and lo...,2815
8,SECTION 1. SHORT TITLE. This Act may be cited ...,Troop Talent Act of 2013 - Directs the Secreta...,Troop Talent Act of 2013,2479
9,SECTION 1. SHORT TITLE. This Act may be cited ...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,947


In [13]:
sample_encode = tokenizer.encode(df_bills.text[0])
decode = tokenizer.decode_tokens_bytes(sample_encode)
decode

[b'SECTION',
 b' ',
 b'1',
 b'.',
 b' SHORT',
 b' TITLE',
 b'.',
 b' This',
 b' Act',
 b' may',
 b' be',
 b' cited',
 b' as',
 b' the',
 b' ``',
 b'National',
 b' Science',
 b' Education',
 b' Tax',
 b' In',
 b'cent',
 b'ive',
 b' for',
 b' Businesses',
 b' Act',
 b' of',
 b' ',
 b'200',
 b'7',
 b"''.",
 b' SEC',
 b'.',
 b' ',
 b'2',
 b'.',
 b' C',
 b'RED',
 b'ITS',
 b' FOR',
 b' CERT',
 b'AIN',
 b' CONTRIBUT',
 b'IONS',
 b' BEN',
 b'EF',
 b'IT',
 b'ING',
 b' SC',
 b'IENCE',
 b',',
 b' TECHNO',
 b'LOGY',
 b',',
 b' ENGINE',
 b'ERING',
 b',',
 b' AND',
 b' MAT',
 b'HE',
 b'MAT',
 b'ICS',
 b' EDUC',
 b'ATION',
 b' AT',
 b' THE',
 b' ELEMENT',
 b'ARY',
 b' AND',
 b' SECOND',
 b'ARY',
 b' SCHOOL',
 b' LEVEL',
 b'.',
 b' (',
 b'a',
 b')',
 b' In',
 b' General',
 b'.--',
 b'Sub',
 b'part',
 b' D',
 b' of',
 b' part',
 b' IV',
 b' of',
 b' sub',
 b'chapter',
 b' A',
 b' of',
 b' chapter',
 b' ',
 b'1',
 b' of',
 b' the',
 b' Internal',
 b' Revenue',
 b' Code',
 b' of',
 b' ',
 b'198',
 b'6',


In [14]:
len(decode)

1466

In [17]:
df_first_10 = df_bills.head(10)
df_first_10

,text,summary,title,n_tokens
0,SECTION 1. SHORT TITLE. This Act may be cited ...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,1466
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,1183
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,937
3,SECTION 1. SHORT TITLE. This Act may be cited ...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,3670
4,SECTION 1. SHORT TITLE. This Act may be cited ...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,1038
5,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...,2026
6,SECTION 1. SHORT TITLE. This Act may be cited ...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...,880
7,SECTION 1. SHORT TITLE. This Act may be cited ...,Race to the Top Act of 2010 - Directs the Secr...,A bill to provide incentives for States and lo...,2815
8,SECTION 1. SHORT TITLE. This Act may be cited ...,Troop Talent Act of 2013 - Directs the Secreta...,Troop Talent Act of 2013,2479
9,SECTION 1. SHORT TITLE. This Act may be cited ...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,947


In [18]:
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),
  api_version = "2024-02-01",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

def generate_embeddings(text, model="text-embedding-3-small"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

df_bills['text-embedding-3-small'] = df_bills["text"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-3-small')) # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model

In [20]:
df_bills

,text,summary,title,n_tokens,text-embedding-3-small
0,SECTION 1. SHORT TITLE. This Act may be cited ...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,1466,"[0.040015269070863724, 0.01991669088602066, 0...."
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,1183,"[0.008059188723564148, 0.035038743168115616, 0..."
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,937,"[0.019029008224606514, 0.028144391253590584, 0..."
3,SECTION 1. SHORT TITLE. This Act may be cited ...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,3670,"[0.035134244710206985, 0.013835246674716473, 0..."
4,SECTION 1. SHORT TITLE. This Act may be cited ...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,1038,"[0.011007638648152351, 0.04204106330871582, 0...."
5,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...,2026,"[0.0026806213427335024, 0.05408391356468201, 0..."
6,SECTION 1. SHORT TITLE. This Act may be cited ...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...,880,"[0.03304169699549675, 0.021111415699124336, 0...."
7,SECTION 1. SHORT TITLE. This Act may be cited ...,Race to the Top Act of 2010 - Directs the Secr...,A bill to provide incentives for States and lo...,2815,"[0.03968443721532822, 0.03420981764793396, 0.0..."
8,SECTION 1. SHORT TITLE. This Act may be cited ...,Troop Talent Act of 2013 - Directs the Secreta...,Troop Talent Act of 2013,2479,"[0.02842569351196289, 0.041829679161310196, 0...."
9,SECTION 1. SHORT TITLE. This Act may be cited ...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,947,"[0.022207090631127357, 0.011338761076331139, 0..."


In [21]:
df_bills.to_csv('bills.csv', index=False)

In [24]:
column_data = df_bills['text-embedding-3-small'].head(10)
for i in column_data:
  print(i)

[0.040015269070863724, 0.01991669088602066, 0.02064424194395542, 0.013095906004309654, 0.027672898024320602, 0.029517756775021553, -0.023814281448721886, 0.02663354016840458, -0.003191152820363641, 0.007678254507482052, 0.05612531304359436, 0.0029524252749979496, 0.031388599425554276, -0.0073274713940918446, 0.024035146459937096, 0.036481454968452454, 0.03003743663430214, 0.01638287492096424, 0.0165127944201231, 0.04100266098976135, 0.03315550833940506, -0.007931598462164402, -0.02450285665690899, 0.03661137446761131, -0.008802060037851334, -0.026321733370423317, -0.02780281752347946, -0.04445852339267731, -0.000530234829057008, -0.019968658685684204, 0.032791733741760254, -0.01183568499982357, -0.007723726332187653, -0.023060748353600502, 0.0023531706538051367, 0.037468843162059784, -0.03567595034837723, 0.045004185289144516, 0.05126631632447243, -0.006833776365965605, -0.006723344791680574, -0.023996170610189438, -0.00904890801757574, 0.03344133123755455, -0.07046844810247421, 0.0065

In [25]:
len(column_data[0])

1536

In [26]:
len(column_data[1])

1536

In [27]:
len(column_data[2])

1536

In [55]:
node = pd.read_csv(os.path.join(os.getcwd(), 'node.csv'), header=0)

node

,1,2,3,4,5,6,7,8,9,10,11,12
0,The user has rated 1 products,the length of the username is 26,the user gives 0 products a score of one,0 products a score of two,0 products a score of three,0 products a score of four and 1 products a s...,min,max and average of ratings are 5.0,5,5,and 5.0 respectively. The number of days betw...,and the user's all reviews are positive.
1,The user has rated 4 products,the length of the username is 17,the user gives 0 products a score of one,1 products a score of two,1 products a score of three,0 products a score of four and 2 products a s...,min,max and average of ratings are 4.0,2,5,and 3.75 respectively. The number of days bet...,and the user's all reviews are positive.
2,The user has rated 2 products,the length of the username is 15,the user gives 0 products a score of one,0 products a score of two,0 products a score of three,2 products a score of four and 0 products a s...,min,max and average of ratings are 4.0,4,4,and 4.0 respectively. The number of days betw...,and the user's all reviews are positive.
3,The user has rated 1 products,the length of the username is 21,the user gives 0 products a score of one,0 products a score of two,0 products a score of three,0 products a score of four and 1 products a s...,min,max and average of ratings are 5.0,5,5,and 5.0 respectively. The number of days betw...,and the user's all reviews are positive.
4,The user has rated 2 products,the length of the username is 18,the user gives 0 products a score of one,0 products a score of two,0 products a score of three,1 products a score of four and 1 products a s...,min,max and average of ratings are 4.5,4,5,and 4.5 respectively. The number of days betw...,and the user's all reviews are positive.
...,...,...,...,...,...,...,...,...,...,...,...,...
11939,The user has rated 2 products,the length of the username is 31,the user gives 0 products a score of one,0 products a score of two,0 products a score of three,0 products a score of four and 2 products a s...,min,max and average of ratings are 5.0,5,5,and 5.0 respectively. The number of days betw...,and the user's all reviews are positive.
11940,The user has rated 1 products,the length of the username is 29,the user gives 1 products a score of one,0 products a score of two,0 products a score of three,0 products a score of four and 0 products a s...,min,max and average of ratings are 1.0,1,1,and 1.0 respectively. The number of days betw...,and the user's all reviews are positive.
11941,The user has rated 1 products,the length of the username is 10,the user gives 0 products a score of one,0 products a score of two,0 products a score of three,0 products a score of four and 1 products a s...,min,max and average of ratings are 5.0,5,5,and 5.0 respectively. The number of days betw...,and the user's all reviews are positive.
11942,The user has rated 1 products,the length of the username is 10,the user gives 0 products a score of one,0 products a score of two,0 products a score of three,0 products a score of four and 1 products a s...,min,max and average of ratings are 5.0,5,5,and 5.0 respectively. The number of days betw...,and the user's all reviews are positive.


In [58]:
node_first_10 = node.head(10)
node_first_10

,1,2,3,4,5,6,7,8,9,10,11,12
0,The user has rated 1 products,the length of the username is 26,the user gives 0 products a score of one,0 products a score of two,0 products a score of three,0 products a score of four and 1 products a s...,min,max and average of ratings are 5.0,5,5,and 5.0 respectively. The number of days betw...,and the user's all reviews are positive.
1,The user has rated 4 products,the length of the username is 17,the user gives 0 products a score of one,1 products a score of two,1 products a score of three,0 products a score of four and 2 products a s...,min,max and average of ratings are 4.0,2,5,and 3.75 respectively. The number of days bet...,and the user's all reviews are positive.
2,The user has rated 2 products,the length of the username is 15,the user gives 0 products a score of one,0 products a score of two,0 products a score of three,2 products a score of four and 0 products a s...,min,max and average of ratings are 4.0,4,4,and 4.0 respectively. The number of days betw...,and the user's all reviews are positive.
3,The user has rated 1 products,the length of the username is 21,the user gives 0 products a score of one,0 products a score of two,0 products a score of three,0 products a score of four and 1 products a s...,min,max and average of ratings are 5.0,5,5,and 5.0 respectively. The number of days betw...,and the user's all reviews are positive.
4,The user has rated 2 products,the length of the username is 18,the user gives 0 products a score of one,0 products a score of two,0 products a score of three,1 products a score of four and 1 products a s...,min,max and average of ratings are 4.5,4,5,and 4.5 respectively. The number of days betw...,and the user's all reviews are positive.
5,The user has rated 1 products,the length of the username is 6,the user gives 0 products a score of one,0 products a score of two,0 products a score of three,0 products a score of four and 1 products a s...,min,max and average of ratings are 5.0,5,5,and 5.0 respectively. The number of days betw...,and the user's all reviews are positive.
6,The user has rated 1 products,the length of the username is 16,the user gives 0 products a score of one,0 products a score of two,1 products a score of three,0 products a score of four and 0 products a s...,min,max and average of ratings are 3.0,3,3,and 3.0 respectively. The number of days betw...,and the user's all reviews are positive.
7,The user has rated 1 products,the length of the username is 12,the user gives 0 products a score of one,0 products a score of two,0 products a score of three,0 products a score of four and 1 products a s...,min,max and average of ratings are 5.0,5,5,and 5.0 respectively. The number of days betw...,and the user's all reviews are positive.
8,The user has rated 4 products,the length of the username is 14,the user gives 1 products a score of one,0 products a score of two,0 products a score of three,1 products a score of four and 2 products a s...,min,max and average of ratings are 4.5,1,5,and 3.75 respectively. The number of days bet...,and the user's all reviews are positive.
9,The user has rated 2 products,the length of the username is 5,the user gives 0 products a score of one,0 products a score of two,0 products a score of three,1 products a score of four and 1 products a s...,min,max and average of ratings are 4.5,4,5,and 4.5 respectively. The number of days betw...,and the user's all reviews are positive.


In [73]:
node_first_10['9'] = node_first_10['9'].astype(str)
node_first_10['10'] = node_first_10['10'].astype(str)

In [74]:
node_first_10['text'] = node_first_10['1'] + '.' + node_first_10['2'] + '.' + node_first_10['3']\
                        + ',' + node_first_10['4'] + ',' + node_first_10['5'] + ',' + node_first_10['6']\
                        + ',' + node_first_10['7'] + ',' + node_first_10['8'] + ', ' + node_first_10['9']\
                        + ', ' + node_first_10['10'] + ',' + node_first_10['11'] + ',' + node_first_10['12']

node_first_10['text']

0    The user has rated 1 products. the length of t...
1    The user has rated 4 products. the length of t...
2    The user has rated 2 products. the length of t...
3    The user has rated 1 products. the length of t...
4    The user has rated 2 products. the length of t...
5    The user has rated 1 products. the length of t...
6    The user has rated 1 products. the length of t...
7    The user has rated 1 products. the length of t...
8    The user has rated 4 products. the length of t...
9    The user has rated 2 products. the length of t...
Name: text, dtype: object

In [75]:
node_first_10.text[0]

"The user has rated 1 products. the length of the username is 26. the user gives 0 products a score of one, 0 products a score of two, 0 products a score of three, 0 products a score of four and 1 products a score of five. The ratio of the user's positive and negative ratings are 1.0 and 0.0. The median, min, max and average of ratings are 5.0, 5, 5, and 5.0 respectively. The number of days between the user's first and last rating is 0. The length of the user's feedback is 13, and the user's all reviews are positive."

In [80]:
for i in range(1,13):
  del node_first_10[f'{i}']
node_first_10

,text
0,The user has rated 1 products. the length of t...
1,The user has rated 4 products. the length of t...
2,The user has rated 2 products. the length of t...
3,The user has rated 1 products. the length of t...
4,The user has rated 2 products. the length of t...
5,The user has rated 1 products. the length of t...
6,The user has rated 1 products. the length of t...
7,The user has rated 1 products. the length of t...
8,The user has rated 4 products. the length of t...
9,The user has rated 2 products. the length of t...


In [82]:
node['9'] = node['9'].astype(str)
node['10'] = node['10'].astype(str)

node['text'] = node['1'] + '.' + node['2'] + '.' + node['3']\
               + ',' + node['4'] + ',' + node['5'] + ',' + node['6']\
               + ',' + node['7'] + ',' + node['8'] + ', ' + node['9']\
               + ', ' + node['10'] + ',' + node['11'] + ',' + node['12']

for i in range(1,13):
  del node[f'{i}']

In [83]:
node.text[0]

"The user has rated 1 products. the length of the username is 26. the user gives 0 products a score of one, 0 products a score of two, 0 products a score of three, 0 products a score of four and 1 products a score of five. The ratio of the user's positive and negative ratings are 1.0 and 0.0. The median, min, max and average of ratings are 5.0, 5, 5, and 5.0 respectively. The number of days between the user's first and last rating is 0. The length of the user's feedback is 13, and the user's all reviews are positive."

In [84]:
node

,text
0,The user has rated 1 products. the length of t...
1,The user has rated 4 products. the length of t...
2,The user has rated 2 products. the length of t...
3,The user has rated 1 products. the length of t...
4,The user has rated 2 products. the length of t...
...,...
11939,The user has rated 2 products. the length of t...
11940,The user has rated 1 products. the length of t...
11941,The user has rated 1 products. the length of t...
11942,The user has rated 1 products. the length of t...


In [85]:
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),
  api_version = "2024-02-01",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

def generate_embeddings(text, model="text-embedding-3-small"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

node['text-embedding-3-small'] = node["text"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-3-small')) # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model

In [88]:
node.to_csv('node_embedding.csv', index=False)

In [87]:
column_data = node['text-embedding-3-small'].head(10)
for i in column_data:
  print(i)

[-0.007397916167974472, -0.0009090512176044285, 0.023260554298758507, 0.025910137221217155, 0.043453168123960495, -0.03966008126735687, 0.032101795077323914, 0.02648189105093479, 0.04406675696372986, 0.03422146290540695, 0.007648929487913847, -0.0850655734539032, -0.05996425822377205, -0.015869610011577606, 0.05301956087350845, 0.03042837604880333, -0.002524076960980892, -0.014921339228749275, -0.07279381901025772, 0.030567826703190804, -0.002992983441799879, 0.022437788546085358, 0.05307534337043762, 0.016803937032818794, 0.02323266491293907, 0.04300692304968834, -0.009266570210456848, 0.006474048364907503, -0.006927266251295805, -0.034249354153871536, -0.014837667346000671, -0.017278073355555534, 0.028615502640604973, 0.010019609704613686, 0.004493833053857088, -0.03438880667090416, 0.021294284611940384, 0.030512046068906784, 0.059350669384002686, -0.011302565224468708, -0.05873708426952362, 0.02405542880296707, 0.004563558846712112, -0.05466509237885475, -0.021015381440520287, -0.02

In [89]:
len(column_data[0])

1536